In [1]:
from ray import Reader
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

from fortnite_python import Fortnite
from fortnite_python.domain import Platform

fortnite = Fortnite('<<Insert Fortnite API Key Here>>')

def get_time_now():
    d = datetime.datetime.now()
    d = datetime.datetime(d.year, d.month, d.day, d.hour, d.minute, d.second)
    return d


def process_file(full_replay_file_name):
    with Reader('input/' + full_replay_file_name) as replay:
        ####### REPLAY #######  
        print(replay.stats)
        print(replay.team_stats)

        replay_file = replay.src.split('/')[1]
        file_splits = replay_file.replace('-','.').split('.')[1:7]
        file_splits = [int(x) for x in file_splits]
        replay_datetime = datetime.datetime(file_splits[0],file_splits[1],file_splits[2], file_splits[3], file_splits[4],file_splits[5])
        replay_dict = vars(replay.header)
        replay_dict.update({
            'file': replay_file,
            'datetime': replay_datetime
            })
        df_replay = pd.DataFrame([replay_dict])
        df_replay = df_replay[['file', 'datetime', 'fortnite_version', 'game_sub', 'release', 'season', 'server_side_version']]

        ####### ELIMS #######     
        elims = []
        for elim in replay.eliminations:
            elims.append(vars(elim))


        df_elims = pd.DataFrame(elims)
        df_elims['file'] = replay_file

        df_elims.shape

        df_elims['seconds'] = df_elims['time'].apply(lambda x: x.to_pydatetime().timestamp())
        df_elims.tail()
        df_elims = df_elims[['file', 'eliminated', 'eliminator', 'gun_type', 'knocked', 'time', 'seconds', 'weapon']]
        df_elims.head()

        ####### PLAYERS #######     
        players_names = set(list(df_elims.eliminated) + list(df_elims.eliminator))
        len(players_names)

        from fortnite_python.base import UnknownPlayerError
        import time

        player_stats = []
        for player_name in list(players_names):
            for platform in [Platform.PC, Platform.PSN, Platform.XBOX]:
                repeat = 1
                failed = True
                while failed and repeat <= 3:
                    print(player_name, platform, repeat)
                    try:
                        search_player_name = player_name + '/' # to deal with names with trailing dot. e.g. 'Joe.''
                        player = fortnite.player(search_player_name, platform) 
                        df_player = pd.DataFrame(player.life_time_stats)
                        player_stats_dict = {
                            'player_name': player_name,
                            'platform': platform.value,
                            'retrieved_date': get_time_now()
                        }
                        player_stats_dict.update(df_player.set_index('key')['value'].to_dict())
                        player_stats.append(player_stats_dict)
                        failed = False
                    except UnknownPlayerError as e:
                        print(type(e))
                        failed = False
                    except Exception as e:
                        print(type(e))
                        failed = True
                    time.sleep(2)
                    repeat += 1

        df_players = pd.DataFrame(player_stats)

        df_players['file'] = replay_file
        df_players = df_players[['file', 'player_name', 'platform', 'retrieved_date', 'K/d', 'Kills', 'Matches Played', 'Score', 'Top 10', 'Top 12s',
               'Top 25s', 'Top 3s', 'Top 5s', 'Top 6s', 'Win%', 'Wins']]
        df_players.columns = [s.replace(' ','_').replace('K/d','kd_rate').replace('%','_pct').lower() for s in df_players.columns]
        # Fix number types
        number_text_columns = ['kd_rate', 'kills', 'matches_played', 'score', 'top_10', 'top_12s', 'top_25s', 'top_3s', 'top_5s', 'top_6s', 'win_pct', 'wins']
        def to_number(s):
            return pd.to_numeric(s.replace('%','').replace(',',''))
        for col in number_text_columns:
            df_players[col] = df_players[col].apply(to_number)

        ##### SAVE IT ALL ####
        os.makedirs(os.path.dirname("output/"+replay_file), exist_ok=True)
        df_players.to_csv("output/"+replay_file+"/players.csv", index=False)
        df_replay.to_csv("output/"+replay_file+"/replay.csv", index=False)
        df_elims.to_csv("output/"+replay_file+"/eliminations.csv", index=False)


In [2]:
replays = os.listdir("input")
print(replays)
for full_replay_file_name in replays:
    print(full_replay_file_name)
    process_file(full_replay_file_name)

['UnsavedReplay-2019.01.17-19.39.29.replay', 'UnsavedReplay-2019.01.17-21.36.44.replay', 'UnsavedReplay-2019.01.17-21.14.38.replay', 'UnsavedReplay-2019.01.18-21.32.41.replay', 'UnsavedReplay-2019.01.18-19.13.33.replay', 'UnsavedReplay-2019.01.17-20.01.44.replay', 'UnsavedReplay-2019.01.17-20.30.00.replay', 'UnsavedReplay-2019.01.18-19.46.29.replay', 'UnsavedReplay-2019.01.18-20.43.50.replay', 'UnsavedReplay-2019.01.17-20.50.58.replay', 'UnsavedReplay-2019.01.18-20.12.58.replay']
UnsavedReplay-2019.01.17-19.39.29.replay
{'unknown': 0, 'accuracy': 0, 'assists': 0, 'eliminations': 0, 'weapon_damage': 0, 'other_damage': 0, 'revives': 0, 'damage_taken': 114, 'damage_structures': 0, 'materials_gathered': 0, 'materials_used': 0, 'total_traveled': 0}
{'unknown': 0, 'position': 63, 'total_players': 94}
Dom only Platform.PC 1
Dom only Platform.PSN 1
Dom only Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
KILLERFINN4000 Platform.PC 1
KILLERFINN4000 Platform.PSN 1
KILLERF

<class 'fortnite_python.exceptions.UnknownPlayerError'>
Baccuh Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
PappasBros123 Platform.PC 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
PappasBros123 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
PappasBros123 Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
esra star Platform.PC 1
esra star Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
esra star Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
avonice Platform.PC 1
avonice Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
avonice Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
back2DAlobbyBOII Platform.PC 1
back2DAlobbyBOII Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
back2DAlobbyBOII Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Salvatore Edward Platform.PC 1
Sal

ｔimｔimｓam Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ʟᴜᴄᴀsツ Platform.PC 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ʟᴜᴄᴀsツ Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ʟᴜᴄᴀsツ Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
IQ_KILLAH829 Platform.PC 1
IQ_KILLAH829 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
IQ_KILLAH829 Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
dogzillar Platform.PC 1
dogzillar Platform.PSN 1
dogzillar Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
taobaoshopper Platform.PC 1
taobaoshopper Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
taobaoshopper Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
SUPASPIDERPIG Platform.PC 1
SUPASPIDERPIG Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
SUPASPIDERPIG Platform.XBOX 1
<class 'fortnit

ERROR:ray.logging:Missing weapon type 16


UnsavedReplay-2019.01.17-21.36.44.replay
{'unknown': 0, 'accuracy': 0, 'assists': 0, 'eliminations': 0, 'weapon_damage': 0, 'other_damage': 0, 'revives': 0, 'damage_taken': 100, 'damage_structures': 0, 'materials_gathered': 0, 'materials_used': 0, 'total_traveled': 0}
{'unknown': 0, 'position': 64, 'total_players': 95}
FoghornLeghorn93 Platform.PC 1
FoghornLeghorn93 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
FoghornLeghorn93 Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
hundyhard Platform.PC 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
hundyhard Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
hundyhard Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
xNBnatoes Platform.PC 1
xNBnatoes Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
xNBnatoes Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
SlinKyZ.YT Platform.PC 1
SlinKyZ.YT Platf

...

<class 'fortnite_python.exceptions.UnknownPlayerError'>
FrazDaddy_Twitch Platform.PC 1
FrazDaddy_Twitch Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
FrazDaddy_Twitch Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ʞɐɹʌɐɔuǝʞ Platform.PC 1
ʞɐɹʌɐɔuǝʞ Platform.PSN 1
ʞɐɹʌɐɔuǝʞ Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ElHitMan44.4 Platform.PC 1
ElHitMan44.4 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ElHitMan44.4 Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
TOXIC34466 Platform.PC 1
TOXIC34466 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
TOXIC34466 Platform.XBOX 1
Brooksy73 Platform.PC 1
Brooksy73 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Brooksy73 Platform.XBOX 1
skitzey._.YT Platform.PC 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
skitzey._.YT Platform.PSN 1
<class 'fortnite_python.exception

ERROR:ray.logging:Missing weapon type 40
ERROR:ray.logging:Missing weapon type 16
ERROR:ray.logging:Missing weapon type 16


UnsavedReplay-2019.01.18-20.12.58.replay
{'unknown': 0, 'accuracy': 0, 'assists': 0, 'eliminations': 0, 'weapon_damage': 0, 'other_damage': 0, 'revives': 0, 'damage_taken': 110, 'damage_structures': 0, 'materials_gathered': 0, 'materials_used': 0, 'total_traveled': 0}
{'unknown': 0, 'position': 29, 'total_players': 97}
ttv.kynanOCE Platform.PC 1
ttv.kynanOCE Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
ttv.kynanOCE Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
xxZ0MBI3xx Platform.PC 1
xxZ0MBI3xx Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
xxZ0MBI3xx Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
crenex19 Platform.PC 1
crenex19 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
crenex19 Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
MTS - Billson Platform.PC 1
MTS - Billson Platform.PSN 1
MTS - Billson Platform.XBOX 1
<class 'fortnite_pytho

TTVBeadyEyes Platform.PC 1
TTVBeadyEyes Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
TTVBeadyEyes Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
MxDaddyK Platform.PC 1
MxDaddyK Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
MxDaddyK Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Linden ツ Platform.PC 1
Linden ツ Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Linden ツ Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Youtube_Sxlty Platform.PC 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Youtube_Sxlty Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Youtube_Sxlty Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
CS Chimp Platform.PC 1
CS Chimp Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
CS Chimp Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError

<class 'fortnite_python.exceptions.UnknownPlayerError'>
Zerodestroyer45 Platform.PC 1
Zerodestroyer45 Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
Zerodestroyer45 Platform.XBOX 1
Anima. Platform.PC 1
Anima. Platform.PSN 1
Anima. Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
SamJettRed Platform.PC 1
SamJettRed Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
SamJettRed Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
KKK LoadedDuck Platform.PC 1
KKK LoadedDuck Platform.PSN 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
KKK LoadedDuck Platform.XBOX 1
<class 'fortnite_python.exceptions.UnknownPlayerError'>
